# 使用LangChain、Activeloop的Deep Lake和GPT4对Twitter算法源代码进行分析
在本教程中，我们将使用LangChain + Activeloop的Deep Lake和GPT4来分析Twitter算法的代码库。

In [ ]:
# 使用python3的pip命令安装langchain、deeplake（包括enterprise版本）、openai和tiktoken，并且升级它们到最新版本
!python3 -m pip install --upgrade langchain 'deeplake[enterprise]' openai tiktoken

定义OpenAI嵌入，Deep Lake多模态向量存储API并进行身份验证。有关Deep Lake的完整文档，请参阅[文档](https://docs.activeloop.ai/)和[API参考](https://docs.deeplake.ai/en/latest/)。

如果您想创建自己的数据集并发布它，请在Deep Lake中进行身份验证。您可以从[平台](https://app.activeloop.ai)获取API密钥。

In [1]:
import getpass
import os

from langchain_community.vectorstores import DeepLake  # 导入DeepLake向量存储库
from langchain_openai import OpenAIEmbeddings  # 导入OpenAIEmbeddings库

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")  # 设置OpenAI API密钥
activeloop_token = getpass.getpass("Activeloop Token:")  # 获取Activeloop令牌
os.environ["ACTIVELOOP_TOKEN"] = activeloop_token  # 设置Activeloop令牌

In [2]:
# 导入OpenAIEmbeddings类
from openai import OpenAIEmbeddings

# 创建一个OpenAIEmbeddings对象，传入空的disallowed_special参数
embeddings = OpenAIEmbeddings(disallowed_special=())

为了避免一些仓库在使用tiktoken时出现`Exception: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte`的异常，需要设置disallowed_special=()。

### 1. 对代码库进行索引（可选）
您可以直接跳过此部分，直接开始使用已经索引的数据集。首先，我们将克隆存储库，然后解析和分块代码库，并使用OpenAI索引。

In [3]:
# 克隆指定的github仓库
!git clone https://github.com/twitter/the-algorithm # 替换成你想要克隆的任何仓库

Cloning into 'the-algorithm'...
remote: Enumerating objects: 9142, done.
remote: Counting objects: 100% (2438/2438), done.
remote: Compressing objects: 100% (1662/1662), done.
remote: Total 9142 (delta 597), reused 2349 (delta 593), pack-reused 6704
Receiving objects: 100% (9142/9142), 7.67 MiB | 33.29 MiB/s, done.
Resolving deltas: 100% (2818/2818), done.


加载存储库中的所有文件。

In [4]:
# 导入必要的模块
import os
from langchain_community.document_loaders import TextLoader

# 设置根目录
root_dir = "./the-algorithm"
docs = []

# 遍历根目录下的所有文件
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        try:
            # 使用TextLoader加载文本文件
            loader = TextLoader(os.path.join(dirpath, file), encoding="utf-8")
            # 将加载的文档添加到docs列表中
            docs.extend(loader.load_and_split())
        except Exception:
            # 如果出现异常则跳过当前文件
            pass

然后，对文件进行分块。

In [5]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

Created a chunk of size 2549, which is longer than the specified 1000
Created a chunk of size 2095, which is longer than the specified 1000
Created a chunk of size 1983, which is longer than the specified 1000
Created a chunk of size 1531, which is longer than the specified 1000
Created a chunk of size 1102, which is longer than the specified 1000
Created a chunk of size 1012, which is longer than the specified 1000
Created a chunk of size 1981, which is longer than the specified 1000
Created a chunk of size 1022, which is longer than the specified 1000
Created a chunk of size 1134, which is longer than the specified 1000
Created a chunk of size 1532, which is longer than the specified 1000
Created a chunk of size 1056, which is longer than the specified 1000
Created a chunk of size 1515, which is longer than the specified 1000
Created a chunk of size 2591, which is longer than the specified 1000
Created a chunk of size 1957, which is longer than the specified 1000
Created a chunk of s

执行索引操作。这将花费大约4分钟来计算嵌入并上传到Activeloop。然后你可以将数据集发布为公开的。

In [6]:
username = "<用户名或组织名>"  # 替换为你在 app.activeloop.ai 上的用户名
db = DeepLake(
    dataset_path=f"hub://{username}/twitter-algorithm",  # 使用指定的用户名构建数据集路径
    embedding=embeddings,  # 使用指定的嵌入模型
)
db.add_documents(texts)  # 向数据库中添加文本数据

Deep Lake Dataset in hub://adilkhan/twitter-algorithm already exists, loading from the storage
Batch upload: 31310 samples are being uploaded in 32 batches of batch size 1000


Evaluating ingest: 28%|██▊       | 9/32 [00:50<02:09Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-ciTI4gyz985F6II5qjFfD4Gf on tokens per min. Limit: 1000000 / min. Current: 851354 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Evaluating ingest: 38%|███▊      | 12/32 [01:13<02:09Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-ciTI4gyz985F6II5qjFfD4Gf on tokens per min. Limit: 1000000 / min. Current: 836180 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Evaluating ingest: 41%|████      | 13/32 [01:29<02:43Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seco

Dataset(path='hub://adilkhan/twitter-algorithm', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype       shape       dtype  compression
  -------   -------     -------     -------  ------- 
 embedding  generic  (31310, 1536)   None     None   
    ids      text     (31310, 1)      str     None   
 metadata    json     (31310, 1)      str     None   
   text      text     (31310, 1)      str     None   


['081a3beb-3a8d-11ee-b840-13905694aaaf',
 '081a3bec-3a8d-11ee-b840-13905694aaaf',
 '081a3bed-3a8d-11ee-b840-13905694aaaf',
 '081a3bee-3a8d-11ee-b840-13905694aaaf',
 '081a3bef-3a8d-11ee-b840-13905694aaaf',
 '081a3bf0-3a8d-11ee-b840-13905694aaaf',
 '081a3bf1-3a8d-11ee-b840-13905694aaaf',
 '081a3bf2-3a8d-11ee-b840-13905694aaaf',
 '081a3bf3-3a8d-11ee-b840-13905694aaaf',
 '081a3bf4-3a8d-11ee-b840-13905694aaaf',
 '081a3bf5-3a8d-11ee-b840-13905694aaaf',
 '081a3bf6-3a8d-11ee-b840-13905694aaaf',
 '081a3bf7-3a8d-11ee-b840-13905694aaaf',
 '081a3bf8-3a8d-11ee-b840-13905694aaaf',
 '081a3bf9-3a8d-11ee-b840-13905694aaaf',
 '081a3bfa-3a8d-11ee-b840-13905694aaaf',
 '081a3bfb-3a8d-11ee-b840-13905694aaaf',
 '081a3bfc-3a8d-11ee-b840-13905694aaaf',
 '081a3bfd-3a8d-11ee-b840-13905694aaaf',
 '081a3bfe-3a8d-11ee-b840-13905694aaaf',
 '081a3bff-3a8d-11ee-b840-13905694aaaf',
 '081a3c00-3a8d-11ee-b840-13905694aaaf',
 '081a3c01-3a8d-11ee-b840-13905694aaaf',
 '081a3c02-3a8d-11ee-b840-13905694aaaf',
 '081a3c03-3a8d-

`可选`: 您还可以将Deep Lake的托管张量数据库用作托管服务，并在那里运行查询。为此，需要在创建向量存储时将运行时参数指定为 {'tensor_db': True}。此配置使得可以在托管张量数据库上执行查询，而不是在客户端上执行。需要注意的是，此功能不适用于本地存储或内存中存储的数据集。如果已经在托管张量数据库之外创建了向量存储，可以按照规定的步骤将其转移到托管张量数据库中。

In [7]:
# 将下面的代码翻译成中文，并在代码上方添加适当的注释，保留原始代码。

# 用户名 = "davitbun"  # 请替换为您在 app.activeloop.ai 上的用户名
# db = DeepLake(
#     dataset_path=f"hub://{username}/twitter-algorithm",  # 数据集路径，使用您的用户名替换
#     embedding_function=embeddings,  # 嵌入函数，根据具体情况进行替换
#     runtime={"tensor_db": True}  # 运行时参数，根据具体情况进行替换
# )
# db.add_documents(texts)  # 添加文档，根据具体情况进行替换

### 2. 在 Twitter 上进行问答的算法代码库
首先加载数据集，构建检索器，然后构建对话链。

In [8]:
# 创建一个DeepLake对象，指定数据集路径、只读模式和嵌入式算法
db = DeepLake(
    dataset_path=f"hub://{username}/twitter-algorithm",  # 数据集路径
    read_only=True,  # 只读模式
    embedding=embeddings,  # 嵌入式算法
)

Deep Lake Dataset in hub://adilkhan/twitter-algorithm already exists, loading from the storage


In [9]:
# 创建一个检索器对象
retriever = db.as_retriever()

# 设置检索器的搜索参数，距离度量为余弦相似度
retriever.search_kwargs["distance_metric"] = "cos"

# 设置检索器的搜索参数，获取前100个结果
retriever.search_kwargs["fetch_k"] = 100

# 设置检索器的搜索参数，启用最大边际相关性
retriever.search_kwargs["maximal_marginal_relevance"] = True

# 设置检索器的搜索参数，返回前10个相关结果
retriever.search_kwargs["k"] = 10

你也可以使用[Deep Lake filters](https://docs.deeplake.ai/en/latest/deeplake.core.dataset.html#deeplake.core.dataset.Dataset.filter)来指定用户定义的函数。

In [10]:
def filter(x):
    # 根据源代码进行过滤
    if "com.google" in x["text"].data()["value"]:
        return False

    # 根据路径进行过滤，例如扩展名
    metadata = x["metadata"].data()["value"]
    return "scala" in metadata["source"] or "py" in metadata["source"]


### 打开以下代码以进行自定义过滤
# retriever.search_kwargs['filter'] = filter



In [12]:
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0613")  # 切换到 'gpt-4' 可选的模型
qa = ConversationalRetrievalChain.from_llm(model, retriever=retriever)  # 使用model和retriever创建ConversationalRetrievalChain实例

In [ ]:
# 定义问题列表
questions = [
    "What does favCountParams do?",
    "is it Likes + Bookmarks, or not clear from the code?",
    "What are the major negative modifiers that lower your linear ranking parameters?",
    "How do you get assigned to SimClusters?",
    "What is needed to migrate from one SimClusters to another SimClusters?",
    "How much do I get boosted within my cluster?",
    "How does Heavy ranker work. what are it’s main inputs?",
    "How can one influence Heavy ranker?",
    "why threads and long tweets do so well on the platform?",
    "Are thread and long tweet creators building a following that reacts to only threads?",
    "Do you need to follow different strategies to get most followers vs to get most likes and bookmarks per tweet?",
    "Content meta data and how it impacts virality (e.g. ALT in images).",
    "What are some unexpected fingerprints for spam factors?",
    "Is there any difference between company verified checkmarks and blue verified individual checkmarks?",
]
# 创建对话历史列表
chat_history = []

# 遍历问题列表
for question in questions:
    # 调用qa函数获取问题的答案
    result = qa({"question": question, "chat_history": chat_history})
    # 将问题和答案添加到对话历史列表
    chat_history.append((question, result["answer"]))
    # 打印问题和答案
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

**问题**：`favCountParams`是做什么的？

**答案**：`favCountParams`是一个可选的`ThriftLinearFeatureRankingParams`实例，表示与排名过程中的“喜欢数”特征相关的参数。它用于控制在排名推文时喜欢数特征的权重。喜欢数是推文被用户标记为喜欢的次数，被认为是排名推文中的重要信号。通过使用`favCountParams`，系统可以在计算推文的最终排名分数时调整喜欢数的重要性。

**问题**：是喜欢数+书签，还是代码中不清楚？

**答案**：从提供的代码来看，不清楚喜欢数指标是否由喜欢和书签的总和确定。代码中提到了喜欢数，但没有明确说明它是如何根据喜欢和书签计算的。

**问题**：降低线性排名参数的主要负面修饰符是什么？

**答案**：在给定的代码中，降低线性排名参数的主要负面修饰符包括：

1. `scoringData.querySpecificScore`：这个分数调整是基于特定于查询的信息。如果它的值为负数，将降低线性排名参数。

2. `scoringData.authorSpecificScore`：这个分数调整是基于特定于作者的信息。如果它的值为负数，也会降低线性排名参数。

请注意，我无法提供有关这些负面修饰符确切计算的更多信息，因为没有提供确定它们的代码。

**问题**：如何被分配到SimClusters？

**答案**：通过在生产者-生产者相似性图上运行的基于Metropolis-Hastings采样的社区检测算法来分配到SimClusters。这个图是通过计算每个生产者的关注者之间的余弦相似性分数创建的。该算法识别具有相似关注者的生产者的社区或簇，并采用参数*k*来指定要检测的社区数量。

在社区检测之后，不同的用户和内容被表示为这些识别的社区（SimClusters）中的稀疏、可解释的向量。生成的SimClusters嵌入可以用于各种推荐任务。

**问题**：从一个SimClusters迁移到另一个SimClusters需要什么？

**答案**：要从一个SimClusters表示迁移到另一个，可以按照以下一般步骤进行：

1. **准备新表示**：使用任何必要的更新或更改在聚类算法、相似度度量或其他模型参数中创建新的SimClusters表示。确保这个新表示根据需要正确存储和索引。

2. **更新相关代码和配置**：修改相关代码和配置文件，以引用新的SimClusters表示。这可能涉及更新路径或数据集名称以指向新表示，以及根据需要更改代码以使用新的聚类方法或相似度函数。

3. **测试新表示**：在将更改部署到生产环境之前，彻底测试新的SimClusters表示，以确保其有效性和稳定性。这可能涉及运行离线作业，如候选生成和标记候选，验证输出，以及在评估环境中使用评估工具（如TweetSimilarityEvaluationAdhocApp）测试新表示。

4. **部署更改**：一旦新表示经过测试和验证，部署更改到生产环境。这可能涉及创建一个zip文件，上传到打包程序，然后使用Aurora进行调度。确保监视系统，以确保在表示之间实现平稳过渡，并验证新表示是否按预期用于推荐。

5. **监控和评估新表示**：在部署新表示后，继续监视其性能和对推荐的影响。注意任何改进或出现的问题，并准备根据需要对新表示进行迭代。始终确保结果和性能指标与系统的目标和目标一致。

**问题**：在我的集群内，我会得到多少提升？

**答案**：在SimClusters表示中，不可能确定您的内容在集群内得到的确切提升数量，没有关于您的内容及其参与度指标的具体数据。然而，喜欢分数和关注分数以及其他参与信号和SimCluster计算的结合，会影响内容的提升。

**问题**：Heavy ranker是如何工作的？它的主要输入是什么？

**答案**：Heavy Ranker是一个在推荐算法中排名和评分候选者中起关键作用的机器学习模型。它的主要目的是预测用户与推文互动或在平台上与其他用户连接的可能性。

Heavy Ranker的主要输入包括：

1. 静态特征：这些特征可以直接从推文中计算，例如它是否有URL、是否有卡片、是否有引用等。这些特征由索引摄入器生成，当推文生成并存储在索引中时。

2. 实时特征：这些每条推文的特征在推文被索引后可能会发生变化。它们主要包括社交互动，如转推次数、喜欢次数、回复次数以及一些与后续活动计算的垃圾邮件信号。信号摄入器是Heron拓扑的一部分，处理多个事件流以收集和计算这些实时特征。

3. 用户表特征：这些每个用户的特征是从用户服务写入的流中获得的。这个输入用于存储稀疏的实时用户信息，稍后通过查找推文的作者来评分。

4. 搜索上下文特征：这些特征代表当前搜索者的上下文，如他们的UI语言、内容消费和当前时间（暗示）。它们与推文数据结合，计算用于评分的一些特征。

这些输入然后由Heavy Ranker处理，根据候选者的相关性和用户参与的可能性对其进行评分和排名。

**问题**：如何影响Heavy ranker？

**答案**：要影响Heavy Ranker的输出或内容的排名，考虑以下行动：

1. 提高内容质量：创建高质量且引人入胜的内容，与用户相关、信息丰富且有价值。高质量的内容更有可能获得积极的用户参与，而Heavy Ranker在排名内容时会考虑这一点。

2. 增加用户参与：鼓励用户通过喜欢、转推、回复和评论与内容互动。更高的参与水平可能导致在Heavy Ranker的输出中更好的排名。

3. 优化用户资料：用户的声誉，基于其关注者数量和关注者与关注者比例等因素，可能影响其内容的排名。通过关注相关用户、保持合理的关注者与关注者比例并与关注者互动来保持良好的声誉。

4. 提高内容可发现性：在推文中使用相关关键词、标签和提及，使用户更容易找到并与您的内容互动。这种增加的可发现性可能有助于通过Heavy Ranker提高您的内容的排名。

5. 利用多媒体内容：尝试不同的内容格式，如视频、图片和GIF，这可能吸引用户的注意力并增加参与度，从而通过Heavy Ranker获得更好的排名。

6. 用户反馈：监控并回应对您的内容的反馈。积极的反馈可能提高您的排名，而负面反馈则提供了学习和改进的机会。

请注意，Heavy Ranker使用一系列机器学习模型和各种特征来排名内容。虽然上述行动可能有助于影响排名，但由于排名过程由复杂算法确定，并随时间演变，因此不能保证结果。